In [1]:
# %pip install --upgrade pip
# %pip install --disable-pip-version-check \
#     torch==1.13.1 \
#     torchdata==0.5.1 --quiet

# %pip install \
#     transformers==4.27.2 \
#     datasets==2.11.0 --quiet

In [2]:
# %pip install -U datasets
# %pip install --upgrade transformers

## Importing libraries

In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

## Summarize Dialogue without Prompt Engineering
Using pre-trained LLM FLAN-T5 from Hugging Face and uploading dialouges from Hugging face datasets

In [4]:
dataset = load_dataset("knkarthick/dialogsum")

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [9]:
print(dataset['test'][0])

{'id': 'test_0_1', 'dialogue': "#Person1#: Ms. Dawson, I need you to take a dictation for me.\n#Person2#: Yes, sir...\n#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?\n#Person2#: Yes, sir. Go ahead.\n#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.\n#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?\n#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.\n#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.\n#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time

In [12]:
# printing some dialogues with their baseline summaries

example_indices = [160, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print('INPUT DIALOGUE: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE: 
#Person1#: Did you hear about Lulu?
#Person2#: No, what?
#Person1#: She and Vic broke up and now she ' s asked for a transfer.
#Person2#: Get out of here! I didn ' t even know they were dating!
#Person1#: No one really did. They were very discreet and professional at the office.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY
#Person1# and #Person2# are talking about Lulu and Vic's breakup.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example  2
-------------------------------------------------------------------------------------------

Load the FLAN-T5 model creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method

In [13]:
model_name = 'google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Downloading the tokenizer for FLAN-T5 model using AutoTokenizer.from_pretrained() method.

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Testing the tokenizer

In [16]:
sentence = 'What time is it, Ash?'

sentence_encoded = tokenizer(sentence, return_tensors='pt')

print(sentence_encoded)

{'input_ids': tensor([[ 363,   97,   19,   34,    6, 7153,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [17]:
sentence_decoded = tokenizer.decode(
    sentence_encoded['input_ids'][0],
    skip_special_tokens=True
)

print(sentence_decoded)

What time is it, Ash?


## Base LLM Summaries without <b>Prompt Engineering</b>

In [20]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL_GENERATION:\n{output}')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: Did you hear about Lulu?
#Person2#: No, what?
#Person1#: She and Vic broke up and now she ' s asked for a transfer.
#Person2#: Get out of here! I didn ' t even know they were dating!
#Person1#: No one really did. They were very discreet and professional at the office.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and #Person2# are talking about Lulu and Vic's breakup.
---------------------------------------------------------------------------------------------------
MODEL_GENERATION:
Person1#: I don 't know how they got there.
---------------------------------------------------------------------------------------------------
Example  2
--------------------------------

## Summarize Dialogue with an Instruction Prompt

### Zero Shot Inference with an Instruction Prompt
Convert a dialogue and convert it into an instruction prompt

In [22]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""
    
    # input constructed prompt instead of the dialogue
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL_GENERATION:\n{output}')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Did you hear about Lulu?
#Person2#: No, what?
#Person1#: She and Vic broke up and now she ' s asked for a transfer.
#Person2#: Get out of here! I didn ' t even know they were dating!
#Person1#: No one really did. They were very discreet and professional at the office.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and #Person2# are talking about Lulu and Vic's breakup.
---------------------------------------------------------------------------------------------------
MODEL_GENERATION:
Lulu and Vic broke up and now they're asking for a transfer.
---------------------------------------------------------------------------

Better for example 1 but not much better, and did not work for example 2

In [25]:
# trying different prompt

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue

{dialogue}

What was going on?:
"""
    
    # input constructed prompt instead of the dialogue
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL_GENERATION:\n{output}')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue

#Person1#: Did you hear about Lulu?
#Person2#: No, what?
#Person1#: She and Vic broke up and now she ' s asked for a transfer.
#Person2#: Get out of here! I didn ' t even know they were dating!
#Person1#: No one really did. They were very discreet and professional at the office.

What was going on?:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and #Person2# are talking about Lulu and Vic's breakup.
---------------------------------------------------------------------------------------------------
MODEL_GENERATION:
Lulu and Vic broke up and now they are asking for a transfer.
--------------------------------------------------------------------------------------------

Still not a very great version

In [26]:
# trying one more prompt

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
You are a professional summarizer.
Summarize the following conversation in one statement.
The dialogue is as follows:
{dialogue}

Summary:
"""
    
    # input constructed prompt instead of the dialogue
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=100,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL_GENERATION:\n{output}')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

You are a professional summarizer.
Summarize the following conversation in one statement.
The dialogue is as follows:
#Person1#: Did you hear about Lulu?
#Person2#: No, what?
#Person1#: She and Vic broke up and now she ' s asked for a transfer.
#Person2#: Get out of here! I didn ' t even know they were dating!
#Person1#: No one really did. They were very discreet and professional at the office.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and #Person2# are talking about Lulu and Vic's breakup.
---------------------------------------------------------------------------------------------------
MODEL_GENERATION:
Lulu and Vic broke up and now they are dating.
----------

Nope, this didn't work either.

## Summarize Dialogue with One Shot and Few Shot Inference

### One Shot Inference 
Build a function that takes a list of example_indices_full, and generates a prompt with examples

In [27]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # the stop sequence '{summary}\n\n\n' is important for FLAN-T5

        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}
"""
        
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']

        prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
        return prompt

Construct the prompt to perform one shot inference:

In [32]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full=example_indices_full, example_index_to_summarize=example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need

In [29]:
# one shot inference

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}\n')
print(dash_line)
print(f"MODEL GENERATION SUMMARY: \n{output}")

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION SUMMARY: 
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


This is a little better from zero shot inference

### Few Shot Inference
lets add a few more examples

In [39]:
new_example_indices_full = [80, 100, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full=new_example_indices_full, example_index_to_summarize=example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May, can you help me take all these things to the living room?
#Person2#: Yes, madam.
#Person1#: Ask Daniel to give you a hand?
#Person2#: No, mom, I can manage it by myself. His help just causes more trouble.

What was going on?
Mom asks May to help to prepare for the picnic and May agrees.

Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to y

In [40]:
# few shot inference

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}\n')
print(dash_line)
print(f"MODEL GENERATION SUMMARY: \n{output}")

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION SUMMARY: 
#Person1 wants to upgrade his system and hardware.


Not actually better, kind of similar to one-shot inference

## Generative Configuration Parameters for Inference

Tweaking configuration parameters of the generate() method to see different output from the LLM. 

do_sample, top_k, top_p, temperature

In [46]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY: \n{summary}\n')
print(dash_line)
print(f"MODEL GENERATION SUMMARY: \n{output}")

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION SUMMARY: 
#Person1 recommends that you upgrade your system and hardware.ra explains to the other two that you'd probably need a better processor, more memory, and a faster modem.ra offers a
